In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy
from scipy.ndimage import gaussian_filter1d
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error # mse

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import os 
import glob
import cv2
import itertools

from dataloader import *


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# LSTM Module
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm_acc = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm_gyr = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        self.reg_module1 = nn.Sequential(
            nn.Linear(hidden_size*2, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 8),
            nn.ReLU(inplace=True),
            nn.Linear(8, 1)
        )
        

    def forward(self, acc, gyr): 
        
        # 다음 학습에 영향을 주지 않기 위해 초기 h_0과 c_0 초기화
        h0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)

        _, (h_acc, _) = self.lstm_acc(acc, (h0, c0))
        _, (h_gyr, _) = self.lstm_gyr(gyr, (h0, c0))
    
        inputs_concat = torch.cat((h_acc.view(-1, hidden_size), h_gyr.view(-1, hidden_size)), dim=1)
        out_lstm = self.reg_module1(inputs_concat)
        
        return out_lstm

In [3]:
file_path = "D:\gait_dataset/salted/*"
dataset = Gait_Dataset_Salted(file_path)
val_percent = 0.2
n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val
train, val = random_split(dataset, [n_train, n_val])

In [4]:
train_loader = torch.utils.data.DataLoader(train,
                                           batch_size=128,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(val,
                                         batch_size=128,
                                         shuffle=False)

In [5]:
input_size = 300
hidden_size = 64
num_layers = 1

model = LSTM(input_size, hidden_size, num_layers).to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_epochs = 1000

loss_function = nn.MSELoss()

In [6]:
# Set fixed random number seed
torch.manual_seed(7777)

# Early Stopping을 위한 변수
best = 100
converge_cnt = 0

# Run Training loop
for epoch in range(0, n_epochs) :
    # Set current loss value 
    tot_trn_loss = 0.0
    
    # Train Mode
    model.train()
    
    # Iterate over the DataLoader for training data 
    for i, data in enumerate(train_loader) :
        inputs_acc, inputs_gyr, targets = data
        inputs_acc, inputs_gyr, targets = inputs_acc.float(), inputs_gyr.float(), targets.float()
        inputs_acc = inputs_acc.to(device)
        inputs_gyr = inputs_gyr.to(device)
        targets = targets.reshape(-1, 1)
        targets = targets.to(device)

        # 순전파 
        outputs = model(inputs_acc, inputs_gyr)
        
        # Loss 계산
        loss = loss_function(outputs, targets)
        
        # Zero the gradients 
        optimizer.zero_grad()
        # 역전파
        loss.backward()
        # Perform optimization 
        optimizer.step() 
        
        # Print statistics
        tot_trn_loss += loss.item()
        
    # Evaluation Mode
    model.eval()
    
    tot_val_loss = 0
    val_epoch_loss = []

    with torch.no_grad() :
        for i, data in enumerate(val_loader):
            inputs_acc, inputs_gyr, targets = data
            inputs_acc, inputs_gyr, targets = inputs_acc.float(), inputs_gyr.float(), targets.float()
            inputs_acc = inputs_acc.to(device)
            inputs_gyr = inputs_gyr.to(device)
            targets = targets.reshape(-1, 1)
            targets = targets.to(device)
                        
            # 순전파 
            outputs = model(inputs_acc, inputs_gyr)
            
            # Batch 별 Loss 계산
            loss = loss_function(outputs, targets)
            tot_val_loss += loss.item()            
            

    # Epoch 별 Loss
    trn_loss = tot_trn_loss / len(train_loader)
    val_loss = tot_val_loss / len(val_loader)
    val_epoch_loss.append(val_loss)
    
    
    print("Epoch : {}/{}, Train Loss : {:.6f}, Valid Loss {:.6f}".format(epoch+1, n_epochs,
                                                                                       trn_loss, val_loss))
    
    # Early Stopping
    if val_loss < best:
        best = np.mean(val_loss)
        converge_cnt = 0
    else:
        converge_cnt += 1
    
    if converge_cnt > 50:
        print('Early stopping')
        break
    
#     print("Epoch : {}/{} Epoch Loss : {:.6f}".format(epoch+1, n_epochs, current_loss / len(trainloader.dataset)))
print(min(val_epoch_loss))

Epoch : 1/1000, Train Loss : 17843.514242, Valid Loss 17913.157878
Epoch : 2/1000, Train Loss : 17781.104004, Valid Loss 17854.750651
Epoch : 3/1000, Train Loss : 17651.265869, Valid Loss 17553.068685
Epoch : 4/1000, Train Loss : 17006.748820, Valid Loss 16404.546224
Epoch : 5/1000, Train Loss : 15074.377401, Valid Loss 13396.409505
Epoch : 6/1000, Train Loss : 10882.086792, Valid Loss 7934.862386
Epoch : 7/1000, Train Loss : 4946.141968, Valid Loss 2148.812703
Epoch : 8/1000, Train Loss : 883.482466, Valid Loss 292.360769
Epoch : 9/1000, Train Loss : 319.906573, Valid Loss 296.552882
Epoch : 10/1000, Train Loss : 311.939814, Valid Loss 288.987289
Epoch : 11/1000, Train Loss : 308.403503, Valid Loss 288.523125
Epoch : 12/1000, Train Loss : 308.735537, Valid Loss 288.258649
Epoch : 13/1000, Train Loss : 307.852030, Valid Loss 288.429888
Epoch : 14/1000, Train Loss : 307.999543, Valid Loss 288.560135
Epoch : 15/1000, Train Loss : 308.453004, Valid Loss 288.235163
Epoch : 16/1000, Train L

Epoch : 133/1000, Train Loss : 18.365688, Valid Loss 17.241023
Epoch : 134/1000, Train Loss : 17.608108, Valid Loss 17.924470
Epoch : 135/1000, Train Loss : 18.295034, Valid Loss 19.855055
Epoch : 136/1000, Train Loss : 17.951356, Valid Loss 15.586173
Epoch : 137/1000, Train Loss : 17.452861, Valid Loss 17.763008
Epoch : 138/1000, Train Loss : 19.200506, Valid Loss 16.247705
Epoch : 139/1000, Train Loss : 17.878322, Valid Loss 15.831816
Epoch : 140/1000, Train Loss : 16.838532, Valid Loss 15.987575
Epoch : 141/1000, Train Loss : 16.967364, Valid Loss 16.042442
Epoch : 142/1000, Train Loss : 17.753614, Valid Loss 15.596292
Epoch : 143/1000, Train Loss : 17.448234, Valid Loss 16.144645
Epoch : 144/1000, Train Loss : 16.891270, Valid Loss 14.996863
Epoch : 145/1000, Train Loss : 17.288381, Valid Loss 16.601041
Epoch : 146/1000, Train Loss : 16.908607, Valid Loss 15.533703
Epoch : 147/1000, Train Loss : 17.599434, Valid Loss 24.289843
Epoch : 148/1000, Train Loss : 18.575021, Valid Loss 15

Epoch : 265/1000, Train Loss : 13.743089, Valid Loss 17.239477
Epoch : 266/1000, Train Loss : 13.222885, Valid Loss 14.431031
Epoch : 267/1000, Train Loss : 11.328628, Valid Loss 14.229660
Epoch : 268/1000, Train Loss : 11.822112, Valid Loss 12.475748
Epoch : 269/1000, Train Loss : 11.229802, Valid Loss 16.664233
Epoch : 270/1000, Train Loss : 12.396696, Valid Loss 13.034094
Epoch : 271/1000, Train Loss : 12.053738, Valid Loss 11.507158
Epoch : 272/1000, Train Loss : 11.142594, Valid Loss 11.810723
Epoch : 273/1000, Train Loss : 11.681785, Valid Loss 15.219431
Epoch : 274/1000, Train Loss : 13.004852, Valid Loss 12.658012
Epoch : 275/1000, Train Loss : 11.152963, Valid Loss 11.470185
Epoch : 276/1000, Train Loss : 11.253933, Valid Loss 11.685269
Epoch : 277/1000, Train Loss : 11.782667, Valid Loss 11.695464
Epoch : 278/1000, Train Loss : 12.026556, Valid Loss 15.063349
Epoch : 279/1000, Train Loss : 11.187093, Valid Loss 11.255485
Epoch : 280/1000, Train Loss : 10.852469, Valid Loss 12

Epoch : 397/1000, Train Loss : 8.998206, Valid Loss 10.498788
Epoch : 398/1000, Train Loss : 8.981791, Valid Loss 10.742903
Epoch : 399/1000, Train Loss : 8.369495, Valid Loss 10.142751
Epoch : 400/1000, Train Loss : 9.381063, Valid Loss 11.612102
Epoch : 401/1000, Train Loss : 8.138059, Valid Loss 10.558767
Epoch : 402/1000, Train Loss : 8.434222, Valid Loss 10.169509
Epoch : 403/1000, Train Loss : 8.849316, Valid Loss 10.266486
Epoch : 404/1000, Train Loss : 8.864819, Valid Loss 9.903698
Epoch : 405/1000, Train Loss : 8.557919, Valid Loss 13.526413
Epoch : 406/1000, Train Loss : 10.634432, Valid Loss 9.995327
Epoch : 407/1000, Train Loss : 10.207454, Valid Loss 9.965683
Epoch : 408/1000, Train Loss : 8.200171, Valid Loss 10.197138
Epoch : 409/1000, Train Loss : 8.730258, Valid Loss 10.323763
Epoch : 410/1000, Train Loss : 8.910158, Valid Loss 9.946690
Epoch : 411/1000, Train Loss : 8.785566, Valid Loss 11.856901
Epoch : 412/1000, Train Loss : 8.443433, Valid Loss 10.598417
Epoch : 41

Epoch : 531/1000, Train Loss : 6.067839, Valid Loss 10.202530
Epoch : 532/1000, Train Loss : 6.645296, Valid Loss 10.316754
Epoch : 533/1000, Train Loss : 7.235519, Valid Loss 9.773104
Epoch : 534/1000, Train Loss : 7.767439, Valid Loss 9.910338
Epoch : 535/1000, Train Loss : 6.402451, Valid Loss 8.870997
Epoch : 536/1000, Train Loss : 6.598183, Valid Loss 9.661762
Epoch : 537/1000, Train Loss : 6.273912, Valid Loss 9.449165
Epoch : 538/1000, Train Loss : 6.416787, Valid Loss 9.479396
Epoch : 539/1000, Train Loss : 6.243819, Valid Loss 9.230173
Epoch : 540/1000, Train Loss : 6.513553, Valid Loss 9.525838
Epoch : 541/1000, Train Loss : 6.119587, Valid Loss 9.192241
Epoch : 542/1000, Train Loss : 5.912576, Valid Loss 8.953144
Epoch : 543/1000, Train Loss : 6.174207, Valid Loss 10.226153
Epoch : 544/1000, Train Loss : 6.307659, Valid Loss 9.313653
Epoch : 545/1000, Train Loss : 6.239843, Valid Loss 9.064045
Epoch : 546/1000, Train Loss : 6.129379, Valid Loss 9.900979
Epoch : 547/1000, Tra

Epoch : 665/1000, Train Loss : 5.398625, Valid Loss 9.214578
Epoch : 666/1000, Train Loss : 4.937365, Valid Loss 8.887685
Epoch : 667/1000, Train Loss : 5.259057, Valid Loss 11.087280
Epoch : 668/1000, Train Loss : 5.258602, Valid Loss 8.849310
Epoch : 669/1000, Train Loss : 4.963677, Valid Loss 8.678467
Epoch : 670/1000, Train Loss : 5.053885, Valid Loss 8.440699
Epoch : 671/1000, Train Loss : 4.936166, Valid Loss 8.500221
Epoch : 672/1000, Train Loss : 4.714136, Valid Loss 8.369712
Epoch : 673/1000, Train Loss : 5.838861, Valid Loss 8.533288
Epoch : 674/1000, Train Loss : 5.987300, Valid Loss 8.433028
Epoch : 675/1000, Train Loss : 4.699981, Valid Loss 8.960471
Epoch : 676/1000, Train Loss : 4.601401, Valid Loss 8.076967
Epoch : 677/1000, Train Loss : 5.976278, Valid Loss 11.369517
Epoch : 678/1000, Train Loss : 5.290850, Valid Loss 8.276173
Epoch : 679/1000, Train Loss : 4.714567, Valid Loss 8.026692
Epoch : 680/1000, Train Loss : 5.161360, Valid Loss 8.700176
Epoch : 681/1000, Trai

Epoch : 801/1000, Train Loss : 3.973989, Valid Loss 8.197512
Epoch : 802/1000, Train Loss : 3.582781, Valid Loss 7.610136
Epoch : 803/1000, Train Loss : 3.871473, Valid Loss 7.828439
Epoch : 804/1000, Train Loss : 4.752396, Valid Loss 7.932750
Epoch : 805/1000, Train Loss : 4.297970, Valid Loss 8.514554
Epoch : 806/1000, Train Loss : 4.354229, Valid Loss 8.325437
Epoch : 807/1000, Train Loss : 4.601477, Valid Loss 7.759041
Epoch : 808/1000, Train Loss : 3.737310, Valid Loss 7.652018
Epoch : 809/1000, Train Loss : 3.485382, Valid Loss 8.126933
Epoch : 810/1000, Train Loss : 3.882498, Valid Loss 10.041751
Epoch : 811/1000, Train Loss : 4.369005, Valid Loss 8.454534
Epoch : 812/1000, Train Loss : 3.880675, Valid Loss 8.404684
Epoch : 813/1000, Train Loss : 3.930566, Valid Loss 8.199978
Epoch : 814/1000, Train Loss : 3.679973, Valid Loss 7.619406
Epoch : 815/1000, Train Loss : 3.953698, Valid Loss 8.844503
Epoch : 816/1000, Train Loss : 4.349538, Valid Loss 11.050917
Epoch : 817/1000, Trai

Epoch : 937/1000, Train Loss : 2.934275, Valid Loss 7.051701
Epoch : 938/1000, Train Loss : 3.003890, Valid Loss 7.562341
Epoch : 939/1000, Train Loss : 3.297242, Valid Loss 7.363589
Epoch : 940/1000, Train Loss : 3.097085, Valid Loss 7.292940
Epoch : 941/1000, Train Loss : 3.011237, Valid Loss 7.556475
Epoch : 942/1000, Train Loss : 2.950741, Valid Loss 7.680881
Epoch : 943/1000, Train Loss : 3.345149, Valid Loss 7.649918
Epoch : 944/1000, Train Loss : 3.020181, Valid Loss 7.617889
Epoch : 945/1000, Train Loss : 3.001536, Valid Loss 7.345904
Epoch : 946/1000, Train Loss : 2.997783, Valid Loss 7.056120
Epoch : 947/1000, Train Loss : 2.972615, Valid Loss 7.246064
Epoch : 948/1000, Train Loss : 2.818957, Valid Loss 7.482365
Epoch : 949/1000, Train Loss : 2.999246, Valid Loss 7.297876
Epoch : 950/1000, Train Loss : 2.932861, Valid Loss 7.638053
Epoch : 951/1000, Train Loss : 3.005595, Valid Loss 7.516009
Epoch : 952/1000, Train Loss : 2.983567, Valid Loss 7.789769
Epoch : 953/1000, Train 

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# LSTM Module
class LSTM_atn(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM_atn, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm_acc = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm_gyr = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        self.reg_module1 = nn.Sequential(
            nn.Linear(hidden_size*2, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 8),
            nn.ReLU(inplace=True),
            nn.Linear(8, 1)
        )
    
    def attention(self, lstm_output, final_state):
#         merged_state = torch.cat([s for s in final_state], 1)
        merged_state = final_state.squeeze(0).unsqueeze(2)
        weights = torch.bmm(lstm_output, merged_state)
        weights = F.softmax(weights.squeeze(2), dim=1).unsqueeze(2)
        return torch.bmm(torch.transpose(lstm_output, 1, 2), weights).squeeze(2)

    def forward(self, acc, gyr): 
        
        # 다음 학습에 영향을 주지 않기 위해 초기 h_0과 c_0 초기화
        h0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)

        o_acc, (h_acc, _) = self.lstm_acc(acc, (h0, c0))
        o_gyr, (h_gyr, _) = self.lstm_gyr(gyr, (h0, c0))
        
        h_concat = torch.cat((h_acc.view(-1, hidden_size), h_gyr.view(-1, hidden_size)), dim=1)
        o_concat = torch.cat((o_acc, o_gyr), dim=2)
        
        attn_outputs = self.attention(o_concat, h_concat)
    
        out_lstm = self.reg_module1(attn_outputs)
        
        return out_lstm

In [8]:
input_size = 300
hidden_size = 64
num_layers = 1

model_atn = LSTM_atn(input_size, hidden_size, num_layers).to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model_atn.parameters(), lr=learning_rate)
n_epochs = 1000

loss_function = nn.MSELoss()

In [9]:
# Set fixed random number seed
torch.manual_seed(7777)

# Early Stopping을 위한 변수
best = 100
converge_cnt = 0

# Run Training loop
for epoch in range(0, n_epochs) :
    # Set current loss value 
    tot_trn_loss = 0.0
    
    # Train Mode
    model_atn.train()
    
    # Iterate over the DataLoader for training data 
    for i, data in enumerate(train_loader) :
        inputs_acc, inputs_gyr, targets = data
        inputs_acc, inputs_gyr, targets = inputs_acc.float(), inputs_gyr.float(), targets.float()
        inputs_acc = inputs_acc.to(device)
        inputs_gyr = inputs_gyr.to(device)
        targets = targets.reshape(-1, 1)
        targets = targets.to(device)

        # 순전파 
        outputs = model_atn(inputs_acc, inputs_gyr)
        
        # Loss 계산
        loss = loss_function(outputs, targets)
        
        # Zero the gradients 
        optimizer.zero_grad()
        # 역전파
        loss.backward()
        # Perform optimization 
        optimizer.step() 
        
        # Print statistics
        tot_trn_loss += loss.item()
        
    # Evaluation Mode
    model_atn.eval()
    
    tot_val_loss = 0
    val_epoch_loss = []

    with torch.no_grad() :
        for i, data in enumerate(val_loader):
            inputs_acc, inputs_gyr, targets = data
            inputs_acc, inputs_gyr, targets = inputs_acc.float(), inputs_gyr.float(), targets.float()
            inputs_acc = inputs_acc.to(device)
            inputs_gyr = inputs_gyr.to(device)
            targets = targets.reshape(-1, 1)
            targets = targets.to(device)
                        
            # 순전파 
            outputs = model_atn(inputs_acc, inputs_gyr)
            
            # Batch 별 Loss 계산
            loss = loss_function(outputs, targets)
            tot_val_loss += loss.item()            
            

    # Epoch 별 Loss
    trn_loss = tot_trn_loss / len(train_loader)
    val_loss = tot_val_loss / len(val_loader)
    val_epoch_loss.append(val_loss)
    
    print("Epoch : {}/{}, Train Loss : {:.6f}, Valid Loss {:.6f}".format(epoch+1, n_epochs,
                                                                                       trn_loss, val_loss))
    
    # Early Stopping
    if val_loss < best:
        best = np.mean(val_loss)
        converge_cnt = 0
    else:
        converge_cnt += 1
    
    if converge_cnt > 50:
        print('Early stopping')
        break

print(min(val_epoch_loss))
#     print("Epoch : {}/{} Epoch Loss : {:.6f}".format(epoch+1, n_epochs, current_loss / len(trainloader.dataset)))
            

Epoch : 1/1000, Train Loss : 17660.966960, Valid Loss 17644.122721
Epoch : 2/1000, Train Loss : 17266.199707, Valid Loss 16905.772461
Epoch : 3/1000, Train Loss : 15816.544637, Valid Loss 14362.982259
Epoch : 4/1000, Train Loss : 11815.631063, Valid Loss 8613.284180
Epoch : 5/1000, Train Loss : 5051.536891, Valid Loss 1693.002950
Epoch : 6/1000, Train Loss : 606.098094, Valid Loss 337.412420
Epoch : 7/1000, Train Loss : 343.434059, Valid Loss 288.191589
Epoch : 8/1000, Train Loss : 310.666709, Valid Loss 289.290138
Epoch : 9/1000, Train Loss : 308.210809, Valid Loss 288.145910
Epoch : 10/1000, Train Loss : 308.539772, Valid Loss 288.351435
Epoch : 11/1000, Train Loss : 308.318309, Valid Loss 288.458148
Epoch : 12/1000, Train Loss : 308.734711, Valid Loss 288.264303
Epoch : 13/1000, Train Loss : 307.884874, Valid Loss 288.289459
Epoch : 14/1000, Train Loss : 308.066785, Valid Loss 288.490784
Epoch : 15/1000, Train Loss : 308.494948, Valid Loss 288.364845
Epoch : 16/1000, Train Loss : 30

Epoch : 133/1000, Train Loss : 16.157513, Valid Loss 24.262449
Epoch : 134/1000, Train Loss : 16.423825, Valid Loss 15.805951
Epoch : 135/1000, Train Loss : 16.224046, Valid Loss 15.763993
Epoch : 136/1000, Train Loss : 14.941618, Valid Loss 14.211970
Epoch : 137/1000, Train Loss : 15.516154, Valid Loss 19.083103
Epoch : 138/1000, Train Loss : 15.880020, Valid Loss 14.693691
Epoch : 139/1000, Train Loss : 16.284811, Valid Loss 14.384340
Epoch : 140/1000, Train Loss : 14.552223, Valid Loss 14.212931
Epoch : 141/1000, Train Loss : 14.935271, Valid Loss 14.922416
Epoch : 142/1000, Train Loss : 14.649582, Valid Loss 13.645583
Epoch : 143/1000, Train Loss : 14.769726, Valid Loss 15.059640
Epoch : 144/1000, Train Loss : 14.571427, Valid Loss 14.010738
Epoch : 145/1000, Train Loss : 15.577603, Valid Loss 15.435946
Epoch : 146/1000, Train Loss : 14.467370, Valid Loss 15.065392
Epoch : 147/1000, Train Loss : 15.282425, Valid Loss 19.701740
Epoch : 148/1000, Train Loss : 15.817983, Valid Loss 13

Epoch : 265/1000, Train Loss : 11.668904, Valid Loss 14.562171
Epoch : 266/1000, Train Loss : 11.487328, Valid Loss 11.058732
Epoch : 267/1000, Train Loss : 9.902067, Valid Loss 11.031490
Epoch : 268/1000, Train Loss : 9.904469, Valid Loss 11.905921
Epoch : 269/1000, Train Loss : 10.333526, Valid Loss 12.039819
Epoch : 270/1000, Train Loss : 10.879412, Valid Loss 11.227145
Epoch : 271/1000, Train Loss : 10.723051, Valid Loss 10.985297
Epoch : 272/1000, Train Loss : 10.039472, Valid Loss 11.750648
Epoch : 273/1000, Train Loss : 10.438324, Valid Loss 11.112342
Epoch : 274/1000, Train Loss : 10.622888, Valid Loss 11.247583
Epoch : 275/1000, Train Loss : 9.543184, Valid Loss 10.869694
Epoch : 276/1000, Train Loss : 10.145164, Valid Loss 12.027805
Epoch : 277/1000, Train Loss : 10.278276, Valid Loss 11.809650
Epoch : 278/1000, Train Loss : 9.971650, Valid Loss 11.072384
Epoch : 279/1000, Train Loss : 9.870681, Valid Loss 11.041224
Epoch : 280/1000, Train Loss : 9.528111, Valid Loss 10.80881

Epoch : 399/1000, Train Loss : 7.633287, Valid Loss 10.004035
Epoch : 400/1000, Train Loss : 7.498902, Valid Loss 9.388809
Epoch : 401/1000, Train Loss : 7.019390, Valid Loss 10.343939
Epoch : 402/1000, Train Loss : 7.575585, Valid Loss 11.657354
Epoch : 403/1000, Train Loss : 7.912183, Valid Loss 10.015063
Epoch : 404/1000, Train Loss : 6.944044, Valid Loss 11.172003
Epoch : 405/1000, Train Loss : 7.197232, Valid Loss 10.380101
Epoch : 406/1000, Train Loss : 8.351777, Valid Loss 18.003573
Epoch : 407/1000, Train Loss : 7.688339, Valid Loss 11.221991
Epoch : 408/1000, Train Loss : 7.672661, Valid Loss 9.626038
Epoch : 409/1000, Train Loss : 6.962174, Valid Loss 9.515000
Epoch : 410/1000, Train Loss : 7.402101, Valid Loss 11.717601
Epoch : 411/1000, Train Loss : 8.118859, Valid Loss 12.421859
Epoch : 412/1000, Train Loss : 7.572845, Valid Loss 10.430881
Epoch : 413/1000, Train Loss : 7.010349, Valid Loss 9.283987
Epoch : 414/1000, Train Loss : 8.213525, Valid Loss 11.288054
Epoch : 415/

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# LSTM Module
class CNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM_atn, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.cnn_acc = nn.Conv2d(in_channels=3, hidden_size, num_layers, batch_first=True)
        self.lstm_gyr = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        self.reg_module1 = nn.Sequential(
            nn.Linear(hidden_size*2, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 8),
            nn.ReLU(inplace=True),
            nn.Linear(8, 1)
        )
    
    def attention(self, lstm_output, final_state):
#         merged_state = torch.cat([s for s in final_state], 1)
        merged_state = final_state.squeeze(0).unsqueeze(2)
        weights = torch.bmm(lstm_output, merged_state)
        weights = F.softmax(weights.squeeze(2), dim=1).unsqueeze(2)
        return torch.bmm(torch.transpose(lstm_output, 1, 2), weights).squeeze(2)

    def forward(self, acc, gyr): 
        
        # 다음 학습에 영향을 주지 않기 위해 초기 h_0과 c_0 초기화
        h0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)

        o_acc, (h_acc, _) = self.lstm_acc(acc, (h0, c0))
        o_gyr, (h_gyr, _) = self.lstm_gyr(gyr, (h0, c0))
        
        h_concat = torch.cat((h_acc.view(-1, hidden_size), h_gyr.view(-1, hidden_size)), dim=1)
        o_concat = torch.cat((o_acc, o_gyr), dim=2)
        
        attn_outputs = self.attention(o_concat, h_concat)
    
        out_lstm = self.reg_module1(attn_outputs)
        
        return out_lstm